In [1]:
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from config import settings

In [4]:
PROMPT_ANALYZE_RESUME = """
다음은 사용자의 이력서/포트폴리오 요약입니다:

{resume_summary}

이 문서를 바탕으로 사용자가 적합할 수 있는 직무를 1~2개 추천해주세요.
각 직무에 대해, 이 문서가 어떤 강점을 갖고 있는지도 함께 요약해주세요.

출력 형식:
- 추천 직무: OOO, OOO
- 강점 요약: ...
"""

# 1. PDF 로딩 + 벡터화
def load_resume_pdf(file_path: str):
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.split_documents(pages)
    return docs

# 2. 임베딩 + 벡터 저장
def build_vector_db(docs):
    embeddings = AzureOpenAIEmbeddings(
      azure_deployment=settings.EMBEDDING_DEPLOYMENT,
      openai_api_key=settings.EMBEDDING_API_KEY,
      openai_api_version=settings.EMBEDDING_API_VERSION, 
      azure_endpoint=settings.EMBEDDING_ENDPOINT,  
      chunk_size=1_000,
    ) 
    db = FAISS.from_documents(docs, embeddings)
    return db

# 3. 요약 추출 (RAG)
def summarize_resume(db):
    llm = AzureChatOpenAI(
        deployment_name=settings.LLM_DEPLOYMENT,
        openai_api_key=settings.LLM_API_KEY,
        api_version=settings.LLM_API_VERSION,
        azure_endpoint=settings.LLM_ENDPOINT,
        temperature=settings.LLM_TEMPERATURE
    )
    chain = load_qa_chain(llm, chain_type="stuff")
    docs = db.similarity_search("이 이력서를 간단히 요약해주세요.", k=5)
    summary = chain.run(input_documents=docs, question="이 이력서를 요약해주세요.")
    return summary.strip()

# 4. 자동 직무 추천 프롬프트
def suggest_jobs_from_summary(resume_summary):
    llm = AzureChatOpenAI(
        deployment_name=settings.LLM_DEPLOYMENT,
        openai_api_key=settings.LLM_API_KEY,
        api_version=settings.LLM_API_VERSION,
        azure_endpoint=settings.LLM_ENDPOINT,
        temperature=settings.LLM_TEMPERATURE
    )
    prompt = PROMPT_ANALYZE_RESUME.format(resume_summary=resume_summary)
    return llm.predict(prompt).strip()


In [5]:
file_path = "docs/sample.pdf"

docs = load_resume_pdf(file_path)
vector_db = build_vector_db(docs)
summary = summarize_resume(vector_db)


/var/folders/wy/_jj74ny11pd8y1qwn7xj5vc80000gn/T/ipykernel_30671/1702718214.py:24: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(
/var/folders/wy/_jj74ny11pd8y1qwn7xj5vc80000gn/T/ipykernel_30671/1702718214.py:36: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/var/folders/wy/_jj74ny11pd8y1qwn7xj5vc80000gn

In [6]:
print("🔎 요약 결과:\n", summary)


🔎 요약 결과:
 아래는 박수범 님 이력서의 주요 내용을 간략히 정리한 것입니다.

1. 인적 사항  
- 이름: 박수범  
- 연락처: 010-2898-3951 / qkrtnqja97@gmail.com  
- GitHub: https://github.com/qkrtnqja97  
- 생년월일: 1997.02.04  

2. 진로 목표  
- 특정 기술만 다루는 개발자가 아니라, 문제 정의부터 기획·설계·구현·배포까지 책임지는 ‘풀스택’ 개발자로 성장  
- AI 및 SI/웹앱 개발자를 주된 진로로 설정  

3. 주요 경력 및 프로젝트  
- 중앙대학교 산학협력단 도로교통 연구실 (공공 R&D)  
  · Python 기반 도로 균열 예측 시스템 개발  
  · 점토 블록 성능 시각화 도구 및 환경 센서 플랫폼 구현  
  → 데이터 수집·처리·시각화 전 과정을 설계·구현  
- KAIST App 창업 프로젝트  
  · React/React Native 프론트엔드 파트장  
  · UI 설계, API 연동, AAB 빌드·배포 등 전반 업무 주도  
- 캐나다 Tamwood Web Developer Co-op (2023.05~10)  
  · 영어 환경에서 프론트엔드 집중 교육 및 현지 개발자 협업 프로젝트 수행  

4. 보유 기술  
- 언어 및 플랫폼: Python, JavaScript, React, React Native  
- 임베디드·하드웨어: Arduino IDE, ESP32  
- 데이터베이스: MySQL  
- 역량: 데이터 처리·시각화, 실시간 시스템 설계, API 연동, 앱 빌드·배포  

5. 강점 및 입사 후 포부  
- 문제 정의부터 실험·분석·개선까지 전 과정을 경험하며 ‘사용자 중심’ 솔루션 개발  
- 낯선 분야도 빠르게 학습해 기술에 녹여내는 능력  
- 단순 기능 구현을 넘어 ‘제품의 가치’를 고민하고 발전시켜 나가는 개발자 지향


In [7]:
job_suggestion = suggest_jobs_from_summary(summary)
print("\n🎯 추천 직무:\n", job_suggestion)

/var/folders/wy/_jj74ny11pd8y1qwn7xj5vc80000gn/T/ipykernel_30671/1702718214.py:58: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm.predict(prompt).strip()



🎯 추천 직무:
 - 추천 직무: 풀스택 웹앱 개발자, AI/ML 엔지니어  
- 강점 요약:  
  • 풀스택 개발자로서 Python 기반 백엔드 설계·구현부터 React/React Native 프론트엔드·API 연동·앱 배포까지 전 과정을 주도한 경험  
  • 공공 R&D 프로젝트에서 데이터 수집·처리·시각화 및 예측 모델 개발을 수행하며 머신러닝 파이프라인 전반을 경험  
  • Arduino·ESP32 등 임베디드 플랫폼 연동 경험으로 하드웨어·소프트웨어를 아우르는 시스템 설계 역량 보유  
  • 영어권 협업 경험과 빠른 학습력으로 낯선 기술 스택에도 빠르게 적응, 사용자 중심 솔루션을 책임지는 태도


In [ ]:
PROMPT_HITL_FOLLOWUP = """
사용자의 목표는 '{target_job}' 직무로 이력서/포트폴리오를 보완하는 것입니다.
현재 사용자의 문서 요약 내용은 다음과 같습니다:

{resume_summary}

이 문서를 기반으로 했을 때, 직무 적합성을 높이기 위해 사용자가 추가로 제공하면 좋은 정보나 경험은 무엇인가요?
또한 사용자에게 묻기 좋은 질문 1~2개를 작성해주세요.

형식:
- 보완 포인트:
- 사용자에게 할 질문:
"""
def hitl_followup_questions(target_job, resume_summary):
    llm = AzureChatOpenAI(
        deployment_name=settings.LLM_DEPLOYMENT,
        openai_api_key=settings.LLM_API_KEY,
        api_version=settings.LLM_API_VERSION,
        azure_endpoint=settings.LLM_ENDPOINT,
        temperature=settings.LLM_TEMPERATURE
    )
    prompt = PROMPT_HITL_FOLLOWUP.format(target_job=target_job, resume_summary=resume_summary)
    return llm.predict(prompt).strip()


In [11]:
resume_summary = summary
target_job = "ai 개발자"

follow_up_question = hitl_followup_questions(resume_summary, target_job)
print("🤖 HITL 질문 생성 결과:\n", follow_up_question)

🤖 HITL 질문 생성 결과:
 - 보완 포인트:
  1. AI·머신러닝 프로젝트 경험  
     · 수행한 데이터셋, 문제 정의, 모델 아키텍처(예: CNN, RNN, Transformer 등)  
     · 학습·평가 지표(정확도, F1, RMSE 등) 및 성과  
     · 사용한 프레임워크(TensorFlow, PyTorch 등)와 버전  
  2. 모델 배포·운영 경험  
     · 클라우드(AWS/GCP/Azure) 또는 온프레미스 환경에서의 인퍼런스 파이프라인 구축  
     · Docker/Kubernetes를 활용한 컨테이너화 및 CI/CD 파이프라인  
  3. 데이터 수집·전처리 사례  
     · 웹 크롤링, API 연동, 라벨링 자동화 등 구체적인 워크플로우  
     · 데이터 정제·증강 방법과 도구  
  4. 오픈소스·커뮤니티 활동  
     · GitHub에 공개한 AI 관련 레포지토리 링크  
     · Kaggle·논문 리뷰·스터디 그룹 활동 경력  
  5. AI 윤리·보안 고려 사항  
     · 모델 편향성 분석 및 개선 경험  
     · 개인정보 보호·보안 대책 수립 사례

- 사용자에게 할 질문:
  1. 지금까지 AI·머신러닝 프로젝트 중 가장 도전적이었던 과제와, 해결을 위해 사용한 기법 및 결과는 무엇인가요?  
  2. 학습한 모델을 실제 서비스에 배포·운영해본 경험이 있나요? 있다면 사용한 인프라(클라우드·툴)와 CI/CD 절차를 간략히 설명해 주세요.


In [12]:
PROMPT_FINAL_GENERATION = """
다음은 사용자의 이력서 요약 내용입니다:

{resume_summary}

사용자가 추가로 제공한 정보는 다음과 같습니다:
{user_input_detail}

사용자의 요청은 '{target_job}' 직무에 맞게 문서를 보완하는 것입니다.

위의 정보를 기반으로, 해당 직무에 적합하게 이력서 또는 포트폴리오 내용을 수정하거나 새롭게 작성해주세요.
- 실무 중심, 구체적인 성과 중심으로 작성
- 너무 장황하지 않게
- 포맷은 자연스러운 이력서 문장 스타일

결과:
"""
def generate_final_resume_content(target_job, resume_summary, user_input_detail):
    llm = AzureChatOpenAI(
        deployment_name=settings.LLM_DEPLOYMENT,
        openai_api_key=settings.LLM_API_KEY,
        api_version=settings.LLM_API_VERSION,
        azure_endpoint=settings.LLM_ENDPOINT,
        temperature=settings.LLM_TEMPERATURE
    )
    prompt = PROMPT_FINAL_GENERATION.format(
        target_job=target_job,
        resume_summary=resume_summary,
        user_input_detail=user_input_detail
    )
    return llm.predict(prompt).strip()

In [14]:
user_additional_info = "나 ai 관련해서는 스테이블디퓨젼 기반 모델에 다양한 모델을 적용해보고, 실제 원하는 서비스와 가능여부를 판단해봤어. 그리고 요즘에는 모듈러 레그를 사용해보고 있어." \
"실제 배포 경험도 있고, 어플을 출시해봤어.인프라는 aws 기반으로 구축해봤고, kubernetes도 사용해봤어."

final_resume_content = generate_final_resume_content(
    target_job=target_job,
    resume_summary=summary,
    user_input_detail=user_additional_info
)
print("📝 최종 이력서 내용:\n", final_resume_content)

📝 최종 이력서 내용:
 박수범 (AI·풀스택 개발자)  
연락처 010-2898-3951 · qkrtnqja97@gmail.com · GitHub https://github.com/qkrtnqja97  

[핵심 역량]  
• Python 기반 머신러닝·딥러닝 모델 설계·개발·평가·배포 전 과정 수행  
• Stable Diffusion 등 생성형 AI 파이프라인 커스터마이징 및 서비스화 경험  
• AWS(EC2, S3, ECR, EKS) + Kubernetes로 MLOps 환경 구축·운영  
• React/React Native 프론트엔드부터 임베디드(Arduino/ESP32) 연동까지 풀스택 역량  

[주요 프로젝트]  
1. 도로 균열 예측 시스템 (중앙대학교 산학협력단, 2022.03–12)  
 - Python·OpenCV·scikit-learn 기반 데이터 수집·전처리·머신러닝 파이프라인 설계  
 - Random Forest 모델 적용 후 예측 정확도 87% 달성  
 - FastAPI REST API로 서비스화, Docker 이미지화하여 AWS EKS에 배포 (평균 응답 시간 200ms 이하)  
 - D3.js·Vue.js 기반 시각화 대시보드 제작, 사용자 피드백 30건 반영 개선  

2. Stable Diffusion 기반 이미지 생성 서비스 (개인 프로젝트, 2023.07–현재)  
 - Hugging Face 오픈소스 모델에 LoRA·ControlNet 기법 적용, 도메인별 커스텀 튜닝  
 - FastAPI + gRPC로 추론 서버 구현, AWS S3에 모델·데이터 저장, EKS로 오토스케일링 배포  
 - React Native 앱 연동 후 App Store·Play Store 동시 출시, 월간 1,200+건 이미지 생성 요청 처리  

3. 모듈러 Mixture-of-Experts(모듈형 레이어) LLM 실험 (개인 연구)  
 - Transformer 기반 LLM에 MoE 아키텍처 적용, GPU 클러스터 분산 추론 파이프라

In [15]:
print("📝 최종 이력서 내용:\n", final_resume_content)

📝 최종 이력서 내용:
 박수범 (AI·풀스택 개발자)  
연락처 010-2898-3951 · qkrtnqja97@gmail.com · GitHub https://github.com/qkrtnqja97  

[핵심 역량]  
• Python 기반 머신러닝·딥러닝 모델 설계·개발·평가·배포 전 과정 수행  
• Stable Diffusion 등 생성형 AI 파이프라인 커스터마이징 및 서비스화 경험  
• AWS(EC2, S3, ECR, EKS) + Kubernetes로 MLOps 환경 구축·운영  
• React/React Native 프론트엔드부터 임베디드(Arduino/ESP32) 연동까지 풀스택 역량  

[주요 프로젝트]  
1. 도로 균열 예측 시스템 (중앙대학교 산학협력단, 2022.03–12)  
 - Python·OpenCV·scikit-learn 기반 데이터 수집·전처리·머신러닝 파이프라인 설계  
 - Random Forest 모델 적용 후 예측 정확도 87% 달성  
 - FastAPI REST API로 서비스화, Docker 이미지화하여 AWS EKS에 배포 (평균 응답 시간 200ms 이하)  
 - D3.js·Vue.js 기반 시각화 대시보드 제작, 사용자 피드백 30건 반영 개선  

2. Stable Diffusion 기반 이미지 생성 서비스 (개인 프로젝트, 2023.07–현재)  
 - Hugging Face 오픈소스 모델에 LoRA·ControlNet 기법 적용, 도메인별 커스텀 튜닝  
 - FastAPI + gRPC로 추론 서버 구현, AWS S3에 모델·데이터 저장, EKS로 오토스케일링 배포  
 - React Native 앱 연동 후 App Store·Play Store 동시 출시, 월간 1,200+건 이미지 생성 요청 처리  

3. 모듈러 Mixture-of-Experts(모듈형 레이어) LLM 실험 (개인 연구)  
 - Transformer 기반 LLM에 MoE 아키텍처 적용, GPU 클러스터 분산 추론 파이프라

In [17]:
import json

def save_final_output_json(
    target_job: str,
    resume_summary: str,
    user_input_detail: str,
    final_generated_text: str,
    save_path: str = "./final_resume_output.json"
):
    data = {
        "target_job": target_job,
        "resume_summary": resume_summary,
        "user_input_detail": user_input_detail,
        "final_generated_text": final_generated_text
    }

    # 디렉토리 없으면 생성
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ 최종 결과 저장 완료: {save_path}")


In [18]:
# 3. 질문에 대한 답변 생성
save_final_output_json(
    target_job=target_job,
    resume_summary=summary,
    user_input_detail=user_additional_info,
    final_generated_text=final_resume_content,
    save_path="./final_resume_output.json"
) 

✅ 최종 결과 저장 완료: ./final_resume_output.json
